In [2]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

1. Import data and convert the three digit ZIP to a string as to not lose any leading zeros.

In [3]:
dtype_dict = {'Three-Digit ZIP Code': str}

In [4]:
hpi_zip_df = pd.read_excel('../data/hpi_zip3.xlsx', skiprows=6, dtype=dtype_dict)

2. Create dataframe that focuses on the years being studied.

In [5]:
hpi_20s = hpi_zip_df[(hpi_zip_df['Year']>= 2000)&(hpi_zip_df['Year']<=2020)]

3. Change data types in dataframe to be able to use the numbers.

In [6]:
hpi_20s['Annual Change (%)']= pd.to_numeric(hpi_20s['Annual Change (%)'], errors='coerce')
hpi_20s['HPI with 2000 base']= pd.to_numeric(hpi_20s['HPI with 2000 base'], errors='coerce')
hpi_20s['HPI']= pd.to_numeric(hpi_20s['HPI'], errors='coerce')
hpi_20s['one-digit ZIP'] = hpi_20s['Three-Digit ZIP Code'].str[0]

C:\Users\12564\AppData\Local\Temp\ipykernel_1984\3950337914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hpi_20s['Annual Change (%)']= pd.to_numeric(hpi_20s['Annual Change (%)'], errors='coerce')
C:\Users\12564\AppData\Local\Temp\ipykernel_1984\3950337914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hpi_20s['HPI with 2000 base']= pd.to_numeric(hpi_20s['HPI with 2000 base'], errors='coerce')
C:\Users\12564\AppData\Local\Temp\ipykernel_1984\3950337914.py:3: SettingWithCopyWarning: 
A value is tr

4. Create several groupbys to get the means for the years based on the one-digit Zip.

In [7]:
hpi_pct_avg_20s = hpi_20s.groupby(['one-digit ZIP','Year'])['Annual Change (%)'].mean().reset_index()
hpi_avg_20s = hpi_20s.groupby(['one-digit ZIP','Year'])['HPI'].mean().reset_index()
hpi2000base_avg_20s = hpi_20s.groupby(['one-digit ZIP','Year'])['HPI with 2000 base'].mean().reset_index()

5. Create common dataframe with all numbers including percent change since 2000. (All HPI for all regions reset at 100 for year 2000)

In [8]:
base2010=hpi2000base_avg_20s.loc[hpi2000base_avg_20s['Year']==2010,'HPI with 2000 base']

In [9]:
base2010

10     150.990390
31     147.346979
52     144.264396
73     128.020222
94     110.667634
115    136.424884
136    124.230814
157    137.524333
178    138.590000
199    148.669659
Name: HPI with 2000 base, dtype: float64

In [11]:
hpi2000base_avg_20s['HPI']= hpi_avg_20s['HPI']
hpi2000base_avg_20s['Annual Change (%)'] = hpi_pct_avg_20s['Annual Change (%)']
hpi2000base_avg_20s['Percent change since 2000'] =(hpi2000base_avg_20s['HPI with 2000 base']-100)/(100) *100
hpi2000base_avg_20s['Percent change since 2010'] =(hpi2000base_avg_20s['HPI with 2000 base']-base2010)/(base2010) *100
hpi2000base_avg_20s['HPI year by year percent change since 2000'] = hpi2000base_avg_20s['HPI with 2000 base'].pct_change()*100

In [13]:
hpi2000base_avg_20s

one-digit ZIP  Year  HPI with 2000 base          HPI  Annual Change (%)  \
0               0  2000          100.000000   362.011688          10.674545   
1               0  2001          109.081169   396.925455           9.081169   
2               0  2002          119.488961   436.928961           9.494416   
3               0  2003          128.511558   465.643462           7.492987   
4               0  2004          145.091299   524.491154          12.855769   
..            ...   ...                 ...          ...                ...   
205             9  2016          193.916591   979.643977           7.909773   
206             9  2017          206.816705  1043.760909           6.795114   
207             9  2018          222.095568  1122.624886           7.370909   
208             9  2019          230.280000  1157.207045           3.873750   
209             9  2020          236.302386  1177.330341           2.724205   

     Percent change since 2000  Percent change since 2010  \
0                     0.000000                        NaN   
1                     9.081169                        NaN   
2                    19.488961                        NaN   
3                    28.511558                        NaN   
4                    45.091299                        NaN   
..                         ...                        ...   
205                  93.916591                        NaN   
206                 106.816705                        NaN   
207                 122.095568                        NaN   
208                 130.280000                        NaN   
209                 136.302386                        NaN   

     HPI year by year percent change since 2000  
0                                           NaN  
1                                      9.081169  
2                                      9.541328  
3                                      7.550988  
4                                     12.901361  
..                                          ...  
205                                    7.955119  
206                                    6.652403  
207                                    7.387635  
208                                    3.685095  
209                                    2.615245  

[210 rows x 8 columns]

7. Save file as .csv file for comparison.

In [12]:
hpi2000base_avg_20s.to_csv('../data/hpi_2000-2020.csv',index=False)